## EagleMine: Beyond outliers and on to micro-clusters: Vision-guided Anomaly Detection.

**EagleMine** is a novel tree-based mining approach to recognize and summarize the micro-clusters in the histogram.


### Abstract
Given a histogram for millions of points, what patterns exist in the distributions of point characteristics, and how can we detect them and separate anomalies in a way similar to human vision? Hence, we propose a vision guided algorithm, EagleMine, to recognize and summarize point groups in the feature spaces. EagleMine utilizes a water-level tree to capture group structures according to vision-based intuition at multiple resolutions, and adopts statistical hypothesis tests to determine the optimal groups along the tree. Moreover,EagleMine can identify anomalous micro-clusters (i.e., micro-size groups), which exhibit very similar behavior but deviate away from the majority.


### Summary
Inspired by the mechanism of human vision and cognitive system,

- **EagleMine** detects and summarizes micro-clusters (dense blocks) in the histogram with a hierarchical tree structure (WaterLevelTree alg.),and reports the suspiciousness score of each micro-cluster based on the deviation from the normal (TreeExplore alg.).

- For the large graph, the histogram can be constructed with correlated features of graph nodes, and the micro-clusters correspond to node groups, some of them deviating from the majority and contain anomaly / suspicious objects with high probability.

- Correlated features of graph nodes can be: (in / out) Degree, # Triangle, PageRank, Hubness / Authority, Coreness, etc.


### Overview

| <!-- --> | <!-- -->  | <!-- --> 
|:------------------------:|:------------------------:|:------------------------:|
<img src="images/sinaweibo_outd2hub_histogram_label.png" width=250 /> | <img src="images/eaglemine_wlt_1.png" width=400 /> |  <img src="images/eaglemine_wlt_2.png" width=250/> |
<center><b> Histogram on Sina Weibo data </b></center> | <center><b> WaterLevelTree algorithm </b></center> | <center><b> TreeExplore algorithm </b></center> |


In [ ]:
import spartan as st

You can configure the backend to use GPU or CPU only. Default is using backend cpu. 

## Run EagleMine as a single model

### 1. Create an EagleMine model with basic settings for vocabulary based summarization model:  
**_parameter:_**
  - *voctype*: vocabulary type: {'dtmnorm','dmgauss'}. *Default is 'dtmnorm'*.
  - *mode*: The dimensions of features (the histogram). *Default is $2$*.
  - *mix\_comps*: # mixture component for describing the major island. *Default is $2$*.

In [ ]:
voctype = "dtmnorm"
mode, mix_comps = 2, 2
eaglemine = st.EagleMine(voctype=voctype, mode=mode, mix_comps=mix_comps)

### 2. Load data for EagleMine: there are two ways to set the input data, i.e.,
 - [ ] 2.1. load from the off-the-shelf histogram data.
 - [ ] 2.2. construct histogram based on correlated features (to be extracted) of given a graph.

####  2.1 Using the off-the-shelf histogram

- [x] Histogram from file:
 - *infn\_histogram*: Input path of histogram with the format '$(x,y,z, \cdots)$: val', denoting that the cell $(x,y,z, \cdots)$ affiliates with value 'val'.
 - *infn\_node2hcel*: Input path of the file mapping the node to histogram cell.
 - *infn\_hcel2avgfeat*: Input path of the file mapping the histogram cell to the average features and #points

In [ ]:
inpath = "./inputData/"
infn_histogram = inpath + "histogram.out"
infn_node2hcel = inpath + "node2hcel.out"
infn_hcel2avgfeat = inpath + "hcel2avgfeat.out"

In [ ]:
import zipfile
zFile = zipfile.ZipFile("./inputData/eaglemine_data.zip", "r")
for fileM in zFile.namelist(): 
    zFile.extract(fileM, "./inputData")
zFile.close();

In [ ]:
# load histogram data from file
histogram = st.loadFile2Dict(infn_histogram, 2, int, int)
node2hcel = st.loadFile2Dict(infn_node2hcel, 1, int, int)
hcel2avgfeat = st.loadFile2Dict(infn_hcel2avgfeat, 2, int, float)

#### 2.2 Raw graph data for the EagleMine
- [x] Given a graph, extracts correlated features as following examples, and then construct a histogram to feed EagleMine:
   - bipartite graph:  outdegree vs. hubness  (indegree vs. authorty)
   - unipartite graph: degree vs. pagerank

In [ ]:
# load graph data
inpath = "./inputData/"
in_data = inpath + "example_graph.tensor"
tensor_data = st.loadTensor(path = in_data, header=None)
stensor = tensor_data.toSTensor(hasvalue=True)
graph = st.Graph(stensor, bipartite=True, weighted=True, modet=2)

##### 2.2.1 Extract example features
**_parameter:_**
  - *graph*: graph data
  - *feature\_type*: Feature type for the graph node: {'outdegree2hubness', 'indegree2authority', 'degree2pagerank'}. *Default is 'outdegree2hubness'.*
  
**_return:_**
  - *degreeidx*: The index of 'degree' feature in 'feature'. *$=0$*
  - *feature*: Correlated feature of the graph: numpy.ndarray [$f_x$, $f_y$, $f_z$, $\ldots$]

In [ ]:
feature_type = 'outdegree2hubness'  #  "indegree2authority" #
degreeidx, feature = eaglemine.graph2feature(graph, feature_type)

#### 2.2.2 Construct histogram based on the above feature
**_parameter:_**
   - *feature*: Correlated features of the graph
   - *degreeidx*: The index of 'degree' feature in 'feature'. *$=0$*
   - *N\_bins*: The expected number of bins for generating histogram. *Default is $80$*.
   - *base*: The logarithmic base for bucketing the graph features. *Default is $10$*.
   - *mode*: The dimensions of features (the histogram). *Default is $2$*.
   - *verbose*: Whether output some running logs. *Default is $True$*.

**_return_:**
   - *histogram*: (dict) the histogram data: {$(x,y,z, \cdots)$: cnt}.
   - *node2hcel*: (dict) the mapping from graph node id to histogram cell: {node\_id: $(x,y,z, \cdots)$}
   - *hcel2avgfeat*: (dict) the mapping from histogram cell to the its average feature values" {$(x,y,z, \cdots)$: ($\bar{f_x}, \bar{f_y}, \bar{f_z}$, $\ldots$)}

In [ ]:
histogram, node2hcel,hcel2avgfeat = eaglemine.feature2histogram(feature, degreeidx, N_bins=80, base=10, mode=mode, verbose=True)

####  [Optional] 2.2.3 Save generated histogram data
**_parameter:_**
  - *outfn\_histogram*: Output path for the histogram. *Default is $None$*.
  - *outfn\_node2hcel*: Output path for the file mapping the node to histogram cell. *Default is $None$*.
  - *outfn\_hcel2avgfeat*: Output path for the file mapping the histogram cell to the average features and #points. *Default is $None$*.
  - *comments*: The comments (start character) of inputs. *Default is '$\#$'*.
  - *delimiter*: The separator of items in each line of inputs. *Default is '$,$'*.

In [ ]:
# note：make usre ./output dir has been manually created
outpath = "./output/"
outs_histogram = outpath + "histogram.out"
outs_node2hcel = outpath + "node2hcel.out"
outs_hcel2avgfeat = outpath + "hcel2avgfeat.out"
eaglemine.save_histogram(outs_histogram, outs_node2hcel, outs_hcel2avgfeat, comments="#", delimiter=",")

### Feed histogram data to EagleMine model
**_parameter_:**
   - *histogram*: the histogram data.
   - *node2hcel*: the mapping from graph node id to histogram cell.
   - *hcel2avgfeat*: the mapping from histogram cell to the its average feature values".
   - *weighted_ftidx*: The feature index as weight for suspiciousness metric.*Default is $0$*.

In [ ]:
eaglemine.set_histdata(histogram, node2hcel, hcel2avgfeat, weighted_ftidx=0)

### 3. Run the EagleMine model
**_parameter:_**
 - *outs*: Output path for some temporary results.
 - *waterlevel\_step*: Step size for raising the water level. *Default is $0.2$*.
 - *prune\_alpha*: How proportion of pruning for level-tree. *Default is $0.80$*.
 - *min\_pts*: The minimum number of points in a histogram cell. *Default is $20$*.
 - *strictness*: How strict should the anderson-darling test for normality. 0: not at all strict; 4: very strict. *Default is $3$*.
 - *verbose*: Whether output some running logs. *Default is $True$*.

In [ ]:
# note：make usre ./output dir has been manually created
outpath = "./output/"
eaglemine.run(outs=outpath, waterlevel_step=0.2, prune_alpha=0.80, min_pts=20, strictness=3, verbose=True)

In [ ]:
# output model information
eaglemine.dump()

### [Optional] 4. Save results
**_parameter:_**
  - *outfn\_eaglemine*: Output path for the eaglemine data.
  - *outfn\_leveltree*: Output path for the eater-level-tree data. *Default is $None$*.
  - *outfn\_node2label*: Output path for the file mapping the node to the label of cluster. *Default is $None$*.
  - *outfn\_hcel2label*: Output path for the file mapping the histogram cell to the label of cluster. *Default is $None$*.
  - *comments*: The comments (start character) of outputs. *Default is '$\#$'*.
  - *delimiter*: The separator of items in each line of outputs. *Default is '$,$'*.

In [ ]:
outs_eaglemine = outpath + "eaglemine.out"
outs_leveltree = outpath + "waterleveltree.out"
outs_node2label = outpath + "node2label.out"
outs_hcel2label = outpath + "hcel2label.out"
eaglemine.save(outs_eaglemine, outs_leveltree, outs_node2label, outs_hcel2label, comments="#", delimiter=",")

### [Optinal] 5. Result visualization

In [ ]:
from scipy.sparse import csr_matrix

#### Visualize the two-dimensional histogram $\mathcal{H}$
**infn: histogram**: the input histogram file with following format:
  - the 1st three lines started with ‘#’ are comments for some basic information, that is, 1st line shows the shape of 2-dimensional histogram;
  - the 2nd line gives the corresponding real coordinate of each cell coordinate x, and 
  - the 3rd line is the corresponding real coordinate of each cell coordinate y, these two lines are used for visualizing the histogram. 
  - then the followed lines are non-zero data records of the histogram.

In [ ]:
infn_histogram = outpath + "histogram.out"  # inpath + "histogram.out" # 
hist_shape, ticks_dims, hist_arr = st.loadHistogram(infn_histogram)
hist_spm = csr_matrix((hist_arr[:, -1], (hist_arr[:, 0], hist_arr[:, 1])), shape=hist_shape, dtype=int)

In [ ]:
outfn_hist = outpath + "histogram.png"
x_label, y_label = "Hubness", "Out-degree"  #"Authority", "In-degree"  # "PageRank", "Degree"
hfig = st.histogram_viz(hist_spm, ticks_dims[1], ticks_dims[0], outfn_hist, x_label=x_label, y_label=y_label)

#### Visualize clustering result of EagleMine

**infn: hcel2label** (the file for mapping the histogram cell to the label of cluster)

In [ ]:
infn_hcel2label = inpath + "hcel2label.out"  #outpath + "hcel2label.out" # 
hcel2lab = st.loadFile2Dict(infn_hcel2label, 2, int, int)

In [ ]:
outfn_hcls = outpath + "hcluster.png"
hcsl_fig = st.clusters_viz(hcel2lab, outfn_hcls)

### Experiment results 
------

#### Performance on real-world Sina Weibo data (user-msg-retweet)：

| <!-- --> |  <!-- --> |
|:---------------------------:|:---------------------------:|
| <img src="images/sinaweibo_outd2hub_perform.png" width="360"/>  | <img src="images/sinaweibo_outd2hub_histogram_eaglemine.png" width="240"/>|
| <center><b> Anomaly patterns detected by EagleMine </b></center> | <center><b> EagleMine summarizes the histogram consistent with human vision </b></center> |
| <img src="images/sinaweibo_auc.png" width="240"/>  | <img src="images/eaglemine_runtime.png" width="260"/> |
| <center><b>  Anomaly detection performance </b></center>| <center><b> EagleMine is linearly scalable** </b></center> |


** EagleMine\_DM use the 'dmgauss' as vocabulary term

-----

### Cite:
------
1. Wenjie Feng, Shenghua Liu, Christos Faloutsos, Bryan Hooi, Huawei Shen, and Xueqi Cheng. "Beyond Outliers and on to Micro-clusters: Vision-Guided Anomaly Detection". The 23rd Pacific-Asia Conference on Knowledge Discovery and Data Mining, pp. 541--554. 2019, Springer.

    <details>
    <summary><span style="color:blue">click for BibTex...</span></summary>
    
    ```bibtex
    @inproceedings{feng2019beyond,
      title={Beyond Outliers and on to Micro-clusters: Vision-Guided Anomaly Detection},
      author={Wenjie Feng, Shenghua Liu, Christos Faloutsos, Bryan Hooi, Huawei Shen, and Xueqi Cheng},
      booktitle={The 23rd Pacific-Asia Conference on Knowledge Discovery and Data Mining},
      pages={541--554},
      year={2019},
      organization={Springer}
    }
    ```
    </details>
    
2. Wenjie Feng, Shenghua Liu, Christos Faloutsos, Bryan Hooi, Huawei Shen, and Xueqi Cheng. EagleMine: Vision-guided Micro-clusters recognition and collective anomaly detection , Future Generation Computer Systems, Vol 115, Feb 2021, pp.236-250.

    <details>
    <summary><span style="color:blue">click for BibTex...</span></summary>
    
    ```bibtex
    @article{feng2021eaglemine,
      title={EagleMine: Vision-guided Micro-clusters recognition and collective anomaly detection},
      author={Feng, Wenjie and Liu, Shenghua and Faloutsos, Christos and Hooi, Bryan and Shen, Huawei and Cheng, Xueqi},
      journal={Future Generation Computer Systems},
      volume={115},
      pages={236--250},
      year={2021},
      publisher={Elsevier}
    }
    ```
    </details>